In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import bs4 as BeautifulSoup

from main import *

/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# TODO 2: AFEGRIR LA TEMPORADA ALS DATAFRAME DELS LABELS

# TODO 3: Crear dataframe de labels
'''
    1. FGA: 2P, 3P, Ast
    2. REB: Off, Def
    3. Blocks
    4. TOV/STL: # TODO: acabar de veure com ho gestionem pq no hi hagi jugades repetides
'''

# TODO 4: Descarregar vídeos, processat i crear datasets

# TODO 5: Pensar què collons analitzaré jejej

# TODO 6: Obtenir 3D tracking data dels vídeos

# TODO 7: Crear/Finetune d'una xarxa multimodal: Vídeos + 3D tracking data

In [13]:
driver = initialize_driver(season="2023-24")

general_players_table_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table/'

# Create the empty dataframe for the labels, depending on the type of play
play_type_indices = {"FGA": 11, "REB": 21, "TOV": 23, "STL": 24, "BLK": 25}
play_type = 'FGA'

labels_df = create_dataframe(df_type=play_type)
df_path = '/Users/arnaubarrera/Desktop/MSc Computer Vision/TFM/labeled_plays_NBA/holy_grail/FGA_labels.csv'
labels_df.to_csv(df_path, mode='w', index=True, header=True)

for i in range(1, 3):  # Iterate over the list of players

    # Clicar en cierta columna (tipo de jugada) de un jugador en concreto
    try:

        link_xpath = general_players_table_xpath + f'tbody/tr[{i}]/td[{play_type_indices[play_type]}]/a'
        link_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, link_xpath))
        )

        link_url = link_element.get_attribute("href")
        driver.get(link_url)

    except Exception as e:
        print(f"Error: {e}")

    video_display_xpath = '//*[@id="vjs_video_3_html5_api"]'

    # Load all the rows in one page before iterating
    dropdown_xpath = '//*[@class="DropDown_select__4pIg9"]'
    option_value = '-1'

    dropdown_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, dropdown_xpath))
    )
    select = Select(dropdown_element)
    select.select_by_value(option_value)

    rows_number = count_rows_table_plays(driver, table_xpath='//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody')

    for i in range(1, 5):  # Iterate over all the plays of a particular player
        line_play = f'//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/div/div[3]/table/tbody/tr[{i}]/td'

        play = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, line_play))
        )

        columns_play = play.find_elements(By.XPATH, line_play)
        
        row_data = []
        for col in columns_play:
            row_data.append(col.text)
        
        play.click()

        # Verify if the shot is assisted
        ast_xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section/div/main/section[1]/div/div[2]/h2'

        ast_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, ast_xpath))
        )

        ast_present, assisted_by = extract_assist_info(ast_element.text)
        row_data.extend([ast_present, assisted_by])

        # Video display element
        video_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, video_display_xpath))
        )
        video_src = video_element.get_attribute('src')
        row_data.append(video_src) 

        # La fila se agrega al DataFrame
        labels_df.loc[i - 1] = row_data  
    
    # Actualizar el csv con los nuevos datos
    labels_df.to_csv(df_path, mode='a', index=True, header=False)

    # Cuando se ha guardado, click en la flecha de ir para atrás
    driver.back()
    time.sleep(2)


driver.quit()

Cookies banner closed.


In [14]:
df = pd.read_csv(df_path)